In [ ]:
mysql -u root

use dht;
show tables;
desc collect_data;
select * from collect_data;

SET PASSWORD FOR 'root'@'localhost' = PASSWORD('1111');
exit;

mysql -u root -p

In [ ]:
import pymysql
from sense_hat import SenseHat
import time

sense = SenseHat()
conn = pymysql.connect(user='root', host='localhost', passwd='1111', db='dht')

while True:
    try:
        with conn.cursor() as cur:
            temp = sense.get_temperature()
            humid = sense.get_humidity()
            sql = f"insert into collect_data values(now(), %f, %f)"
            cur.execute(sql, (temp, humid))
            conn.commit()
        time.sleep(1)
    except KeyboardInterrupt:
        conn.close()
        break

In [ ]:
from signal import signal, SIGTERM, SIGHUP, pause
from rpi_lcd import LCD
import requests
import re
import time

# 5V 사용.
url = "http://www.kma.go.kr/wid/queryDFSRSS.jsp?zone=2917069500"

lcd = LCD()
safe_exit = lambda signum, frame: exit(1)
signal(SIGTERM, safe_exit)
signal(SIGHUP, safe_exit)

response = requests.get(url)
results = response.content
text = results.decode('utf-8')

p1 = r'<temp>(.*?)</temp>'
p2 = r'<reh>(.*?)</reh>'

m1 = re.match(p1, text)
m2 = re.match(p2, text)

# for match in re.findall(p1, text):
#     print(match)

try:
    if m1 and m2:
        lcd.text(f"temp: {m1.group(1)}", 1)
        lcd.text(f"humid: {m2.group(1)}", 2)
        pause()
except KeyboardInterrupt:
    lcd.clear()

In [ ]:
import smbus
import time
from flask import Flask
import threading

bus = smbus.SMBus(1)
app = Flask(__name__)

address = 0x48
A0 = 0x40
value = 0

def thread():
    while True:
        bus.write_byte(address, A0)
        global value
        value = bus.read_byte(address)

@app.route('/')
def main():
    global value
    return f'''
    {value}
    '''

if __name__ == "__main__":
    th = threading.Thread(target=thread)
    th.start()
    app.run(debug=True, port=80, host='0.0.0.0')

In [ ]:
import RPi.GPIO as GPIO
from flask import Flask

app = Flask(__name__)

# 3V - GPIO 16
LED = 16
GPIO.setmode(GPIO.BCM)
GPIO.setup(LED, GPIO.OUT)

@app.route('/')
def main():
    return 'ㅎㅇ'

@app.route('/on')
def on():
    GPIO.output(LED, GPIO.LOW)
    return 'on'

@app.route('/off')
def off():
    GPIO.output(LED, GPIO.HIGH)
    return 'off'

if __name__ == "__main__":
    app.run(debug=True, port=80, host='0.0.0.0')

In [ ]:
import RPi.GPIO as GPIO
import cv2
from datetime import datetime
import time
import numpy as np

PIR = 26
GPIO.setmode(GPIO.BCM)
GPIO.setup(PIR, GPIO.IN)
# GPIO.setup(PIR, GPIO.IN, pull_up_down=GPIO.PUD_UP)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break   # release, destroy 다 하고 가야되니까 너는 exit ㄴㄴ
        
    resized_frame = cv2.resize(frame, (600, 400))
    
    cv2.imshow('tab name', frame)
    if GPIO.input(PIR) == GPIO.HIGH:
    # if GPIO.input(PIR) == GPIO.LOW:
        cv2.imwrite(f"{datetime.now()}.png", frame)
        
    if (cv2.waitKey(1) & 0xFF) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()